In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import datetime
import os

import pandas as pd
import numpy as np
import boto3
import joblib
import pytz

from mtclient import MTClient
import mtglobals

In [7]:
client = MTClient()

Your account balance is 2470.60


### (a) Get all HITs since `start_cutoff`

Note that this must be a *localized* datetime object (use `mtglobals.localize_datetime()`)

In [9]:
start_cutoff = mtglobals.localize_datetime(datetime.datetime(2022, 9, 20, 0, 0, 0))
all_hits = client.download_all_hits(start_cutoff=start_cutoff, end_cutoff=None)

p2:SNdtMax88dR7p8iivd704GLu1rqP1LTn+zISRTiI4TUGm3BF2KJPSPTEB1dfC5I=
Saving downloaded HIT data to ../results_2stage/all_hit_data.pkl


### (b) Print out info on all stage-1 HITs (subset of HITs scraped in prev step) and specify which one(s) to monitor

In [10]:
[(h['HITId'], h['Title'], h['CreationTime'], h['MaxAssignments']) for h in all_hits if "instantly unlock" in h['Title']]

[('3EHIMLB7GKKSFG8JX9I12SH0EU48H4',
  'Quick 3-question survey about work [<15 seconds], instantly unlock 2nd-stage HIT with higher reward',
  datetime.datetime(2022, 9, 21, 16, 21, 14, tzinfo=tzlocal()),
  1000)]

In [11]:
current_wave_hit_id = "3EHIMLB7GKKSFG8JX9I12SH0EU48H4"

### (c) Check + approve all submissions, and record the worker ids for stage-2 step

In [10]:
s1_subs = client.get_stage1_submissions(current_wave_hit_id)

=====[ Launched Stage 1 HIT: ]=====
HIT launched 2022-01-31 19:10:51-08:00
Status: Reviewable
MaxAssignments: 1000
Num submissions: 1000
=====[ Approving any remaining assignments ]=====
Assignment #0
Approving 3ZY8KE4ISK4JN1JOGDT6WES5ZZYVQQ, for worker A22V0BPWJIVMHW
{'ResponseMetadata': {'RequestId': 'f24a343f-61fb-4831-b9ef-01a0a8d39cc7', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'f24a343f-61fb-4831-b9ef-01a0a8d39cc7', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Wed, 02 Feb 2022 22:56:35 GMT'}, 'RetryAttempts': 0}}
Assignment #1
Approving 3G0WWMR1UWL6FWAFEM7FMXYSDWWQN4, for worker A1PAY3X73PQ16S
{'ResponseMetadata': {'RequestId': '1e3e0c2a-1dcf-4d0b-b4b6-9fc0c1739b9d', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '1e3e0c2a-1dcf-4d0b-b4b6-9fc0c1739b9d', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Wed, 02 Feb 2022 22:56:35 GMT'}, 'RetryAttempts': 0}}
Assignment #2
Approving 3WYP994K18S7H

RequestError: An error occurred (RequestError) when calling the ApproveAssignment operation: This operation can be called with a status of: Submitted (1643842853735)

### Now run 02a_PostStage2.ipynb once all stage-1 HITs submitted